In [ ]:
%cd ..
%reload_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from transformers import pipeline
from src.data import load_data
import re

pd.set_option("max_colwidth", None)

## Explore the cleaned data

In [ ]:
df = load_data()
print(f"Loaded {len(df):,} posts.")
df.head(5)

In [ ]:
df[df.text.str.contains('[LINK]', regex=False)].sample(frac=1.0).head(50)

## Explore the weakly supervised labels

In [ ]:
ws_df = pd.read_parquet("data/processed/data_2022_weakly_supervised.parquet")
ws_df.head()

In [ ]:
ws_df.query("label == 1").head(3)